# 3. Feature Engineering #
## 3.1 Import libraries and load the data from previous step##

In [2]:
import pandas as pd
import numpy as np
# df = pd.read_csv('\Coursera\Advanced_DS_Capstone_02.csv', index_col=[0])
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
df.columns = ["Feature_1", "Feature_2", "Feature_3", "Feature_4", "Label"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
Feature_1    150 non-null float64
Feature_2    150 non-null float64
Feature_3    150 non-null float64
Feature_4    150 non-null float64
Label        150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


## 3.2 Construction of Pipeline modules ##
### 3.2.1 Construction of Indexer ###

In [3]:
from pyspark.ml.feature import StringIndexer

spark_df = sqlContext.createDataFrame(df)
indexer = StringIndexer(inputCol="Label", outputCol="LabelIndex")
indexed = indexer.fit(spark_df).transform(spark_df)
indexed.show()
indexed.select('LabelIndex').distinct().show()

+---------+---------+---------+---------+-----------+----------+
|Feature_1|Feature_2|Feature_3|Feature_4|      Label|LabelIndex|
+---------+---------+---------+---------+-----------+----------+
|      5.1|      3.5|      1.4|      0.2|Iris-setosa|       2.0|
|      4.9|      3.0|      1.4|      0.2|Iris-setosa|       2.0|
|      4.7|      3.2|      1.3|      0.2|Iris-setosa|       2.0|
|      4.6|      3.1|      1.5|      0.2|Iris-setosa|       2.0|
|      5.0|      3.6|      1.4|      0.2|Iris-setosa|       2.0|
|      5.4|      3.9|      1.7|      0.4|Iris-setosa|       2.0|
|      4.6|      3.4|      1.4|      0.3|Iris-setosa|       2.0|
|      5.0|      3.4|      1.5|      0.2|Iris-setosa|       2.0|
|      4.4|      2.9|      1.4|      0.2|Iris-setosa|       2.0|
|      4.9|      3.1|      1.5|      0.1|Iris-setosa|       2.0|
|      5.4|      3.7|      1.5|      0.2|Iris-setosa|       2.0|
|      4.8|      3.4|      1.6|      0.2|Iris-setosa|       2.0|
|      4.8|      3.0|    

### 3.2.2 Construction of OneHotEncoder ###

In [4]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols=["LabelIndex"], outputCols=["LabelVec"])
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.show()
encoded.select('LabelVec').distinct().show()

+---------+---------+---------+---------+-----------+----------+---------+
|Feature_1|Feature_2|Feature_3|Feature_4|      Label|LabelIndex| LabelVec|
+---------+---------+---------+---------+-----------+----------+---------+
|      5.1|      3.5|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      4.9|      3.0|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      4.7|      3.2|      1.3|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      4.6|      3.1|      1.5|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      5.0|      3.6|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      5.4|      3.9|      1.7|      0.4|Iris-setosa|       2.0|(2,[],[])|
|      4.6|      3.4|      1.4|      0.3|Iris-setosa|       2.0|(2,[],[])|
|      5.0|      3.4|      1.5|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      4.4|      2.9|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|
|      4.9|      3.1|      1.5|      0.1|Iris-setosa|       2.0|(2,[],[])|
|      5.4|      3.7|    

### 3.2.3 Construction of VectorAssembler ###

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["Feature_1","Feature_2","Feature_3", "Feature_4"],
                                  outputCol="FeaturesVec")
vectorized = vectorAssembler.transform(encoded)
vectorized.show()

+---------+---------+---------+---------+-----------+----------+---------+-----------------+
|Feature_1|Feature_2|Feature_3|Feature_4|      Label|LabelIndex| LabelVec|      FeaturesVec|
+---------+---------+---------+---------+-----------+----------+---------+-----------------+
|      5.1|      3.5|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.1,3.5,1.4,0.2]|
|      4.9|      3.0|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.9,3.0,1.4,0.2]|
|      4.7|      3.2|      1.3|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.7,3.2,1.3,0.2]|
|      4.6|      3.1|      1.5|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.6,3.1,1.5,0.2]|
|      5.0|      3.6|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.0,3.6,1.4,0.2]|
|      5.4|      3.9|      1.7|      0.4|Iris-setosa|       2.0|(2,[],[])|[5.4,3.9,1.7,0.4]|
|      4.6|      3.4|      1.4|      0.3|Iris-setosa|       2.0|(2,[],[])|[4.6,3.4,1.4,0.3]|
|      5.0|      3.4|      1.5|      0.2|Iris-setosa|       2.0|(2,[],

### 3.2.4 Construction of Normalizer ###

In [6]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

normalizer = Normalizer(inputCol="FeaturesVec", outputCol="FeaturesNorm", p=1.0)
normalized = normalizer.transform(vectorized)
normalized.show()

+---------+---------+---------+---------+-----------+----------+---------+-----------------+--------------------+
|Feature_1|Feature_2|Feature_3|Feature_4|      Label|LabelIndex| LabelVec|      FeaturesVec|        FeaturesNorm|
+---------+---------+---------+---------+-----------+----------+---------+-----------------+--------------------+
|      5.1|      3.5|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.1,3.5,1.4,0.2]|[0.5,0.3431372549...|
|      4.9|      3.0|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.9,3.0,1.4,0.2]|[0.51578947368421...|
|      4.7|      3.2|      1.3|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.7,3.2,1.3,0.2]|[0.5,0.3404255319...|
|      4.6|      3.1|      1.5|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.6,3.1,1.5,0.2]|[0.48936170212765...|
|      5.0|      3.6|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.0,3.6,1.4,0.2]|[0.49019607843137...|
|      5.4|      3.9|      1.7|      0.4|Iris-setosa|       2.0|(2,[],[])|[5.4,3.9,1.7,0

## 3.3 Pipeline creation ##

In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer])
model = pipeline.fit(spark_df)
prediction = model.transform(spark_df)
prediction.show()

+---------+---------+---------+---------+-----------+----------+---------+-----------------+--------------------+
|Feature_1|Feature_2|Feature_3|Feature_4|      Label|LabelIndex| LabelVec|      FeaturesVec|        FeaturesNorm|
+---------+---------+---------+---------+-----------+----------+---------+-----------------+--------------------+
|      5.1|      3.5|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.1,3.5,1.4,0.2]|[0.5,0.3431372549...|
|      4.9|      3.0|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.9,3.0,1.4,0.2]|[0.51578947368421...|
|      4.7|      3.2|      1.3|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.7,3.2,1.3,0.2]|[0.5,0.3404255319...|
|      4.6|      3.1|      1.5|      0.2|Iris-setosa|       2.0|(2,[],[])|[4.6,3.1,1.5,0.2]|[0.48936170212765...|
|      5.0|      3.6|      1.4|      0.2|Iris-setosa|       2.0|(2,[],[])|[5.0,3.6,1.4,0.2]|[0.49019607843137...|
|      5.4|      3.9|      1.7|      0.4|Iris-setosa|       2.0|(2,[],[])|[5.4,3.9,1.7,0

## 3.4 Summary ##
**Finally we now have a pipeline which results a normalized Featuresvector and an OneHotEncoded Vector of the Labels** 